In [16]:
import torch
from sdf_utils import sap
import time
import random
import deep_sdf
from deep_sdf import mesh, metrics, lr_scheduling, plotting, utils, loss
import deep_sdf.workspace as ws
import json
import os
import reconstruct
import numpy as np
import matplotlib.pyplot as plt

In [20]:
def get_spec_with_default(specs, key, default):
    try:
        return specs[key]
    except KeyError:
        return default

In [21]:
experiment_directory = "examples/torus/Run_w_con_loss_one_variable_usual"
specs = ws.load_experiment_specifications(experiment_directory)
data_source = specs["DataSource"]
test_split_file = specs["TestSplit"]
with open(test_split_file, "r") as f:
    test_split = json.load(f)
eval_test_scene_num = get_spec_with_default(specs, "EvalTestSceneNumber", 10)
eval_test_optimization_steps = get_spec_with_default(specs, "EvalTestOptimizationSteps", 1000)
eval_grid_res = get_spec_with_default(specs, "EvalGridResolution", 256)
torus_path = get_spec_with_default(specs, "TorusPath", "/home/jakaria/torus_two_models_data/torus_two/obj_files")

In [22]:
arch = __import__("networks." + specs["NetworkArch"], fromlist=["Decoder"])
latent_size = specs["CodeLength"]

decoder = arch.Decoder(latent_size, **specs["NetworkSpecs"])
decoder = torch.nn.DataParallel(decoder)

saved_model_state = torch.load(
    os.path.join(
        experiment_directory, ws.model_params_subdir, "latest" + ".pth"
    )
)
saved_model_epoch = saved_model_state["epoch"]

decoder.load_state_dict(saved_model_state["model_state_dict"])
decoder = decoder.module.cuda()
decoder.eval()

Decoder(
  (lin0): Linear(in_features=19, out_features=512, bias=True)
  (lin1): Linear(in_features=512, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=493, bias=True)
  (lin4): Linear(in_features=512, out_features=512, bias=True)
  (lin5): Linear(in_features=512, out_features=512, bias=True)
  (lin6): Linear(in_features=512, out_features=512, bias=True)
  (lin7): Linear(in_features=512, out_features=512, bias=True)
  (lin8): Linear(in_features=512, out_features=1, bias=True)
  (relu): ReLU()
  (th): Tanh()
)

In [23]:
eval_test_time_start = time.time()
test_err_sum = 0.
chamfer_dists = []
chamfer_dists_all = []
test_loss_hists = []
mesh_label_names = []
test_latents = []
sap_scores = []
all_labels = []

In [24]:
eval_test_filenames = deep_sdf.data.get_instance_filenames(data_source, test_split)
labels = torch.load(data_source + "/labels.pt")
eval_test_filenames = random.sample(eval_test_filenames, max(eval_test_scene_num, len(eval_test_filenames)))

{'torus_bump_0000': array([0.        , 1.        , 0.46153846, 0.08361204]), 'torus_bump_0001': array([1.        , 1.        , 0.8361204 , 0.64548495]), 'torus_bump_0002': array([0.        , 1.        , 0.96655518, 0.0735786 ]), 'torus_bump_0003': array([1.        , 1.        , 0.98327759, 0.26421405]), 'torus_bump_0004': array([0.        , 1.        , 0.66889632, 0.48494983]), 'torus_bump_0005': array([1.        , 1.        , 0.92976589, 0.54849498]), 'torus_bump_0006': array([0.        , 1.        , 0.85284281, 0.52508361]), 'torus_bump_0007': array([1.        , 1.        , 0.84949833, 0.73578595]), 'torus_bump_0008': array([0.        , 1.        , 0.04682274, 0.17391304]), 'torus_bump_0009': array([1.        , 1.        , 0.56187291, 0.7993311 ]), 'torus_bump_0010': array([0.        , 1.        , 0.62207358, 0.89297659]), 'torus_bump_0011': array([1.        , 1.        , 0.51839465, 0.19732441]), 'torus_bump_0012': array([0.        , 1.        , 0.42474916, 0.92307692]), 'torus_bump

: 

In [7]:
for test_fname in eval_test_filenames:
    save_name = os.path.basename(test_fname).split(".npz")[0]
    label = labels.get(save_name)
    all_labels.append(label)
    mesh_label_names.append(save_name)
    path = os.path.join(experiment_directory, "test_contrastive_one", ws.tb_logs_test_reconstructions, save_name)
    if not os.path.exists(path):
        os.makedirs(path)
        
    test_fpath = os.path.join(data_source, ws.sdf_samples_subdir, test_fname)
    test_sdf_samples = deep_sdf.data.read_sdf_samples_into_ram(test_fpath)
    test_sdf_samples[0] = test_sdf_samples[0][torch.randperm(test_sdf_samples[0].shape[0])]
    test_sdf_samples[1] = test_sdf_samples[1][torch.randperm(test_sdf_samples[1].shape[0])]

    start = time.time()
    print(f"Optimizing latent code for reconstruction of {test_fname}...")
    test_loss_hist, test_latent = reconstruct.reconstruct(
        decoder,
        int(eval_test_optimization_steps),
        latent_size,
        test_sdf_samples,
        0.01,  # [emp_mean,emp_var],
        0.1,
        num_samples=8000,
        lr=5e-3,
        l2reg=True,
        return_loss_hist=True
    )
    
    if not np.isnan(test_loss_hist[-1]):
        test_err_sum += test_loss_hist[-1]
    test_loss_hists.append(test_loss_hist)
    test_latents.append(test_latent)
    print(f"Reconstruction of {test_fname} took {time.time()-start} seconds.")

    print(f"Creating mesh for {test_fname}...")
    start = time.time()
    with torch.no_grad():
        test_mesh = mesh.create_mesh(
            decoder, 
            test_latent, 
            N=eval_grid_res, 
            max_batch=int(2 ** 18), 
            filename=os.path.join(path, f"epoch={saved_model_epoch}"),
            return_trimesh=True,
        )
    print(f"Mesh creation for {test_fname} took {time.time()-start} seconds.")

    if test_mesh is not None:
        gt_mesh_path = f"{torus_path}/{save_name}.obj"
        cd, cd_all = metrics.compute_metric(gt_mesh=gt_mesh_path, gen_mesh=test_mesh, metric="chamfer")
        chamfer_dists.append(cd)
        chamfer_dists_all.append(cd_all)

    del test_sdf_samples, test_mesh

# Calculate SAP scores
sap_score = sap(factors=all_labels, codes=test_latents.detach().cpu().numpy(), continuous_factors=False, regression=False)

if chamfer_dists:
    mlm = torch.mean(torch.norm(torch.cat(test_latents, dim=0), dim=1))
    fig = plotting.plot_train_stats(loss_hists=test_loss_hists, labels=mesh_label_names)
    fig, percentiles = plotting.plot_dist_violin(np.concatenate(chamfer_dists_all, axis=0))
    for p in [75, 90, 99]:
        if p in percentiles:
            print(f"CD Percentile {p}th: {percentiles[p]}")
    
    plt.show(fig)

Optimizing latent code for reconstruction of /home/jakaria/torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/sdf_data/SdfSamples/obj_files/torus_bump_0848.npz...


KeyboardInterrupt: 

In [7]:
for test_fname in eval_test_filenames:
    save_name = os.path.basename(test_fname).split(".npz")[0]
    label = labels.get(save_name)
    all_labels.append(label[0])
    mesh_label_names.append(save_name)
    path = os.path.join(experiment_directory, "test_contrastive_one", ws.tb_logs_test_reconstructions, save_name)
    if not os.path.exists(path):
        os.makedirs(path)
        
    test_fpath = os.path.join(data_source, ws.sdf_samples_subdir, test_fname)
    test_sdf_samples = deep_sdf.data.read_sdf_samples_into_ram(test_fpath)
    test_sdf_samples[0] = test_sdf_samples[0][torch.randperm(test_sdf_samples[0].shape[0])]
    test_sdf_samples[1] = test_sdf_samples[1][torch.randperm(test_sdf_samples[1].shape[0])]

    start = time.time()
    print(f"Optimizing latent code for reconstruction of {test_fname}...")
    test_loss_hist, test_latent = reconstruct.reconstruct(
        decoder,
        int(eval_test_optimization_steps),
        latent_size,
        test_sdf_samples,
        0.01,  # [emp_mean,emp_var],
        0.1,
        num_samples=8000,
        lr=5e-3,
        l2reg=True,
        return_loss_hist=True
    )

    if not np.isnan(test_loss_hist[-1]):
        test_err_sum += test_loss_hist[-1]
    test_loss_hists.append(test_loss_hist)
    test_latents.append(test_latent)
    print(f"Reconstruction of {test_fname} took {time.time()-start} seconds.")

Optimizing latent code for reconstruction of /home/jakaria/torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/sdf_data/SdfSamples/obj_files/torus_bump_0248.npz...
Reconstruction of /home/jakaria/torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/sdf_data/SdfSamples/obj_files/torus_bump_0248.npz took 9.346868991851807 seconds.
Optimizing latent code for reconstruction of /home/jakaria/torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/sdf_data/SdfSamples/obj_files/torus_bump_1540.npz...
Reconstruction of /home/jakaria/torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/sdf_data/SdfSamples/obj_files/torus_bump_1540.npz took 8.992026090621948 seconds.
Optimizing latent code for reconstruction of /home/jakaria/torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/sdf_data/SdfSamples/obj_files/torus_bump_3601.npz...
Reconstruction of /home/jakaria/torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/sdf_data/SdfSa

In [8]:
test_latents = torch.cat(test_latents)
all_labels = np.array(all_labels)
test_latents = test_latents.cpu().detach().numpy()

In [9]:
all_labels = all_labels.reshape(-1, 1)

In [10]:
print(f"Calculating SAP scores...")
print(f"latent size: {test_latents.shape}")
print(f"all_labels size: {all_labels.shape}")
sap_score = sap(factors=all_labels, codes=test_latents, continuous_factors=False, regression=False)

Calculating SAP scores...
latent size: (22, 16)
all_labels size: (22, 1)


In [13]:
print(f"SAP scores (attribute vae): {sap_score}")

SAP scores: 0.045454545454545414


In [11]:
print(f"SAP scores (contrastive one variable): {sap_score}")

SAP scores (contrastive one variable): 0.09090909090909094


In [ ]:
print(all_labels)

In [19]:
latent_mean = np.mean(test_latents, axis=0)
latent_std = np.std(test_latents, axis=0)

print("Mean for each latent variable:", latent_mean)
print("Standard deviation for each latent variable:", latent_std)

Mean for each latent variable: [ 0.11417606  0.06719077  0.04534976  0.09525654 -0.02522508 -0.06407054
 -0.0300089   0.04920398  0.04075307  0.00277608  0.15100327 -0.00095985
 -0.077442   -0.03339227  0.07768717  0.01228157]
Standard deviation for each latent variable: [0.6846151  0.18429531 0.12822926 0.1802207  0.14744131 0.15811083
 0.16598548 0.21823612 0.16990103 0.26519656 0.16460398 0.24282835
 0.2131904  0.13829003 0.20798773 0.15679179]


In [ ]:
# Loop over each latent variable and display the plot
for i in range(test_latents.shape[1]):
    plt.figure()
    plt.hist(test_latents[:, i], bins=50, alpha=0.7, color='blue', edgecolor='black')
    plt.title(f"Distribution of Latent Variable {i}")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    
    # Show the plot inline in the notebook
    plt.show()